In [1]:
import pandas as pd
import sqlite3
import requests
import time
import json
import tarfile
import os
import re
from tqdm import tqdm
from typing import List, Dict, Optional, Tuple
from urllib.parse import urlencode
import logging

In [2]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
def setup_database(conn):
    """Create all necessary tables in the SQLite database."""
    cursor = conn.cursor()
    
    # GO annotations table
    # geneProductId, goId, goName, goAspect, goEvidence, evidenceCode, qualifier, assignedBy, date
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS uniprot_quickgo_annotations (
            uniprot_id TEXT NOT NULL,
            go_id TEXT NOT NULL,
            go_name TEXT,
            go_aspect TEXT,
            go_evidence TEXT,
            evidence_code TEXT,
            qualifier TEXT,
            assigned_by TEXT,
            date_created TEXT,
            UNIQUE(uniprot_id, go_id)
        )
    ''')
    
    # InterPro to GO mapping table
    # InterPro:IPR000009 Protein phosphatase 2A regulatory subunit PR55 > GO:protein phosphatase type 2A complex ; GO:0000159
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS interpro_go_mapping (
            interpro_id TEXT NOT NULL,
            interpro_description TEXT,
            go_id TEXT NOT NULL,
            go_name TEXT,
            UNIQUE(interpro_id, go_id)
        )
    ''')
    
    # PROSITE entries table
    # Uniprot entry|uniprot id/sequence range: Prosite entry name| prosite entry id/score. aligned sequence.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS prosite_entries (
            prosite_id TEXT NOT NULL,
            uniprot_id TEXT NOT NULL,
            prosite_name TEXT,
            uniprot_name TEXT,
            sequence_start INTEGER,
            sequence_end INTEGER,
            score REAL,
            sequence TEXT,
            aligned_sequence TEXT,
            UNIQUE(prosite_id, uniprot_id, sequence_start, sequence_end)
        )
    ''')

    # InterPro to GO mapping table
    # PROSITE:PS00027 HOMEOBOX_1 > GO:regulation of DNA-templated transcription ; GO:0006355
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS prosite_go_mapping (
            prosite_id TEXT NOT NULL,
            prosite_description TEXT,
            go_id TEXT NOT NULL,
            go_name TEXT,
            UNIQUE(prosite_id, go_id)
        )
    ''')
    
    conn.commit()
    logger.info("Database tables created successfully")

In [ ]:
def fetch_go_annotations(conn, uniprot_ids: List[str], batch_size: int = 100):
    """
    Fetch GO annotations for UniProt IDs using QuickGO API.
    
    Args:
        uniprot_ids: List of UniProt IDs
        batch_size: Number of IDs to process in each batch
    """
    base_url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
    
    # Process in batches to avoid overwhelming the API
    for i in range(0, len(uniprot_ids), batch_size):
        page = 1
        while True:
            batch = uniprot_ids[i:i + batch_size]
            logger.info(f"Processing batch {i//batch_size + 1}, IDs {i+1}-{min(i+batch_size, len(uniprot_ids))}")
            
            # Prepare API parameters
            params = {
                'geneProductId': ','.join(batch),
                'limit': 200,  # Maximum results per request
                'includeFields': 'goName',
                'page': page
            }
            
            try:
                response = requests.get(base_url, params=params, timeout=30)
                response.raise_for_status()
                
                data = response.json()
                annotations = data.get('results', [])
                
                # Insert annotations into database
                cursor = conn.cursor()
                for annotation in annotations:
                    geneProductId = annotation.get('geneProductId', '')
                    geneProductId = geneProductId.split(":")[1] if ':' in geneProductId else geneProductId
                    cursor.execute('''
                        INSERT OR IGNORE INTO uniprot_quickgo_annotations 
                        (uniprot_id, go_id, go_name, go_aspect, go_evidence, evidence_code, qualifier, assigned_by, date_created)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                        geneProductId,
                        annotation.get('goId', ''),
                        annotation.get('goName', ''),
                        annotation.get('goAspect', ''),
                        annotation.get('goEvidence', ''),
                        annotation.get('evidenceCode', ''),
                        annotation.get('qualifier', ''),
                        annotation.get('assignedBy', ''),
                        annotation.get('date', '')
                    ))

                conn.commit()
                logger.info(f"Inserted {len(annotations)} annotations from batch")
                
                # Rate limiting to be respectful to the API
                time.sleep(0.1)

                page += 1
                
            except requests.RequestException as e:
                logger.error(f"Error fetching batch {i//batch_size + 1}: {e}")
                break
            except Exception as e:
                logger.error(f"Unexpected error in batch {i//batch_size + 1}: {e}")
                break

In [63]:
def parse_interpro_go_mapping(conn, file_url: str = "https://current.geneontology.org/ontology/external2go/interpro2go"):
    """
    Download and parse InterPro to GO mapping file.
    
    Args:
        file_url: URL of the interpro2go file
    """
    logger.info("Downloading InterPro to GO mapping file")
    
    try:
        response = requests.get(file_url, timeout=60)
        response.raise_for_status()
        
        lines = response.text.strip().split('\n')
        cursor = conn.cursor()
        
        # Pattern to parse lines: InterPro:IPR000009 Description > GO:description ; GO:0000159
        pattern = r'InterPro:(IPR\d+)\s+(.+?)\s+>\s+GO:(.+?)\s+;\s+(GO:\d+)'
        
        parsed_count = 0
        for line in tqdm(lines):
            line = line.strip()
            if line.startswith('!') or not line:  # Skip comments and empty lines
                continue
            
            match = re.match(pattern, line)
            if match:
                interpro_id = match.group(1)
                interpro_desc = match.group(2).strip()
                go_desc = match.group(3).strip()
                go_term = match.group(4)

                # InterPro to GO mapping table
                cursor.execute('''
                    INSERT OR IGNORE INTO interpro_go_mapping 
                    (interpro_id, interpro_description, go_id, go_name)
                    VALUES (?, ?, ?, ?)
                ''', (interpro_id, interpro_desc, go_term, go_desc))
                
                parsed_count += 1
            else:
                logger.warning(f"Could not parse line: {line}")
        
        conn.commit()
        logger.info(f"Parsed and inserted {parsed_count} InterPro-GO mappings")
        
    except requests.RequestException as e:
        logger.error(f"Error downloading InterPro mapping file: {e}")
    except Exception as e:
        logger.error(f"Error parsing InterPro mapping: {e}")

In [114]:
def download_and_extract_prosite(url: str = "https://ftp.expasy.org/databases/prosite/prosite_alignments.tar.gz", 
                                extract_dir: str = "/cta/share/users/prosite_data"):
    """
    Download and extract PROSITE alignments.
    
    Args:
        url: URL of the PROSITE alignments tar.gz file
        extract_dir: Directory to extract files to
    """
    logger.info("Downloading PROSITE alignments")
    
    try:
        # Download the tar.gz file
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()
        
        tar_filename = "prosite_alignments.tar.gz"
        with open(tar_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        logger.info("Extracting PROSITE alignments")
        
        # Extract the tar.gz file
        with tarfile.open(tar_filename, 'r:gz') as tar:
            tar.extractall(path=extract_dir)
        
        # Clean up the downloaded file
        os.remove(tar_filename)
        
        return extract_dir
        
    except Exception as e:
        logger.error(f"Error downloading/extracting PROSITE data: {e}")
        return None

def parse_prosite_entry(filepath: str) -> List[Dict]:
    """
    Parse a single PROSITE MSA file.
    
    Args:
        filepath: Path to the .msa file
        
    Returns:
        List of parsed entries
    """
    entries = []
    
    try:
        with open(filepath, 'r') as f:
            content = f.read()
        
        # Split by entries (starting with >)
        entry_blocks = re.split(r'^>', content, flags=re.MULTILINE)[1:]  # Skip first empty element
        
        for block in entry_blocks:
            lines = block.strip().split('\n')
            if not lines:
                continue
            
            # Parse header line
            header = lines[0]
            # Pattern with optional score: 
            # With score: GIT2_RAT|Q66H91/1-124: ARFGAP|PS50115/23.341
            # Without score: MIC8_TOXGO|Q9BIM7/229-240: ASX_HYDROXYL|PS00010
            header_pattern = r'([^|]+)\|([^/]+)/(\d+)-(\d+):\s*([^|]+)\|([^/]+)(?:/([0-9.]+))?'
            
            match = re.match(header_pattern, header)
            if match:
                uniprot_name = match.group(1)
                uniprot_id = match.group(2)
                seq_start = int(match.group(3))
                seq_end = int(match.group(4))
                prosite_name = match.group(5)
                prosite_id = match.group(6)
                score = float(match.group(7)) if match.group(7) else None
                
                # Extract sequence (remove gaps and dots, join all sequence lines)
                sequence_lines = lines[1:]
                aligned_sequence = ''.join(sequence_lines).replace(' ', '')
                sequence = ''.join(sequence_lines).replace('-', '').replace('.', '').replace(' ', '').upper()
                
                entries.append({
                    'uniprot_name': uniprot_name,
                    'uniprot_id': uniprot_id,
                    'sequence_start': seq_start,
                    'sequence_end': seq_end,
                    'prosite_name': prosite_name,
                    'prosite_id': prosite_id,
                    'score': score,
                    'sequence': sequence,
                    'aligned_sequence': aligned_sequence
                })
            else:
                logger.warning(f"Could not parse header: {header}")

    except Exception as e:
        logger.error(f"Error parsing file {filepath}: {e}")
    
    return entries

def process_prosite_data(conn, extract_dir: str = "/cta/share/users/prosite_data/prosite_alignments"):
    """
    Process all PROSITE MSA files and insert into database.
    
    Args:
        extract_dir: Directory containing extracted PROSITE files
    """
    if not os.path.exists(extract_dir):
        logger.error(f"Extract directory {extract_dir} does not exist")
        return
    
    cursor = conn.cursor()
    total_entries = 0
    
    # Find all .msa files
    msa_files = [f for f in os.listdir(extract_dir) if f.endswith('.msa')]
    logger.info(f"Found {len(msa_files)} PROSITE MSA files")
    
    for i, filename in enumerate(msa_files):
        if i % 100 == 0:
            logger.info(f"Processing file {i+1}/{len(msa_files)}: {filename}")
        
        filepath = os.path.join(extract_dir, filename)
        entries = parse_prosite_entry(filepath) 

        # Insert entries into database
        for entry in entries:
            cursor.execute('''
                INSERT OR IGNORE INTO prosite_entries 
                (prosite_id, uniprot_id, prosite_name, uniprot_name, sequence_start, sequence_end, 
                    score, sequence, aligned_sequence)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                entry['prosite_id'],
                entry['uniprot_id'],
                entry['prosite_name'],
                entry['uniprot_name'],
                entry['sequence_start'],
                entry['sequence_end'],
                entry['score'],
                entry['sequence'],
                entry['aligned_sequence']
            ))
        
        total_entries += len(entries)
        
        # Commit every 100 files
        if i % 100 == 0:
            conn.commit()
    
    conn.commit()
    logger.info(f"Processed {total_entries} PROSITE entries from {len(msa_files)} files")
    

In [ ]:
def parse_prosite_go_mapping(conn, file_url: str = "https://current.geneontology.org/ontology/external2go/prosite2go"):
    """
    Download and parse Prosite to GO mapping file.
    
    Args:
        file_url: URL of the prosite2go file
    """
    logger.info("Downloading Prosite to GO mapping file")
    
    try:
        response = requests.get(file_url, timeout=60)
        response.raise_for_status()
        
        lines = response.text.strip().split('\n')
        cursor = conn.cursor()
        
        # Pattern to parse lines: PROSITE:PS00027 Description > GO:description ; GO:0000159
        pattern = r'PROSITE:(PS\d+)\s+(.+?)\s+>\s+GO:(.+?)\s+;\s+(GO:\d+)'
        
        parsed_count = 0
        for line in tqdm(lines):
            line = line.strip()
            if line.startswith('!') or not line:  # Skip comments and empty lines
                continue
            
            match = re.match(pattern, line)
            if match:
                prosite_id = match.group(1)
                prosite_description = match.group(2).strip()
                go_desc = match.group(3).strip()
                go_term = match.group(4)

                # Prosite to GO mapping table
                cursor.execute('''
                    INSERT OR IGNORE INTO prosite_go_mapping 
                    (prosite_id, prosite_description, go_id, go_name)
                    VALUES (?, ?, ?, ?)
                ''', (prosite_id, prosite_description, go_term, go_desc))
                
                parsed_count += 1
            else:
                logger.warning(f"Could not parse line: {line}")
        
        conn.commit()
        logger.info(f"Parsed and inserted {parsed_count} Prosite-GO mappings")
        
    except requests.RequestException as e:
        logger.error(f"Error downloading Prosite mapping file: {e}")
    except Exception as e:
        logger.error(f"Error parsing Prosite mapping: {e}")

In [3]:
# Connect to DB
db_file = "/cta/share/users/uniprot/human/human.db"
conn = sqlite3.connect(db_file)

In [4]:
uniprot_ids = list(pd.read_sql(f"SELECT Entry as uniprot_id FROM proteins", conn)['uniprot_id'])

In [ ]:
# setup_database(conn)
# fetch_go_annotations(conn, uniprot_ids) --> Takes very long time. Run it in a screen with a script. 
# parse_interpro_go_mapping(conn)
# download_and_extract_prosite()
# process_prosite_data(conn)
# parse_prosite_go_mapping(conn)

In [ ]:
pd.read_sql(f"SELECT * FROM uniprot_quickgo_annotations", conn)

,uniprot_id,go_id,go_name,go_aspect,go_evidence,evidence_code,qualifier,assigned_by,date_created
0,A0A024R1X5,GO:0007040,lysosome organization,biological_process,IEA,ECO:0000265,acts_upstream_of_or_within,Ensembl,20250313
1,A0A024R1X5,GO:0008285,negative regulation of cell population prolife...,biological_process,IEA,ECO:0000265,acts_upstream_of_or_within,Ensembl,20250313
2,A0A024R1X5,GO:0010507,negative regulation of autophagy,biological_process,IEA,ECO:0000265,acts_upstream_of_or_within,Ensembl,20250313
3,A0A024R1X5,GO:0010613,positive regulation of cardiac muscle hypertrophy,biological_process,IEA,ECO:0000265,acts_upstream_of_or_within,Ensembl,20250313
4,A0A024R1X5,GO:0048666,neuron development,biological_process,IEA,ECO:0000265,acts_upstream_of_or_within,Ensembl,20250313
...,...,...,...,...,...,...,...,...,...
1296713,X6RLR1,GO:0005869,dynactin complex,cellular_component,IEA,ECO:0000256,part_of,InterPro,20250429
1296714,X6RLT1,GO:0045892,negative regulation of DNA-templated transcrip...,biological_process,IEA,ECO:0000256,involved_in,InterPro,20250429
1296715,X6RLT1,GO:0005634,nucleus,cellular_component,IEA,ECO:0000501,located_in,UniProt,20250429
1296716,X6RLU5,GO:0006816,calcium ion transport,biological_process,IEA,ECO:0007322,involved_in,UniProt,20250428


In [127]:
pd.read_sql(f"SELECT * FROM interpro_go_mapping", conn)

,interpro_id,interpro_description,go_id,go_name
0,IPR000003,Retinoid X receptor/HNF4,GO:0003677,DNA binding
1,IPR000003,Retinoid X receptor/HNF4,GO:0003707,nuclear steroid receptor activity
2,IPR000003,Retinoid X receptor/HNF4,GO:0008270,zinc ion binding
3,IPR000003,Retinoid X receptor/HNF4,GO:0006355,regulation of DNA-templated transcription
4,IPR000003,Retinoid X receptor/HNF4,GO:0005634,nucleus
...,...,...,...,...
30199,IPR055345,"Large ribosomal subunit protein eL24-related, ...",GO:0003735,structural constituent of ribosome
30200,IPR055346,"SUF system FeS cluster assembly, SufBD",GO:0016226,iron-sulfur cluster assembly
30201,IPR055351,Urocanase,GO:0016153,urocanate hydratase activity
30202,IPR055438,Succinylglutamate desuccinylase/Aspartoacylase...,GO:0016788,"hydrolase activity, acting on ester bonds"


In [129]:
pd.read_sql(f"SELECT * FROM prosite_entries", conn)

,prosite_id,uniprot_id,prosite_name,uniprot_name,sequence_start,sequence_end,score,sequence,aligned_sequence
0,PS51299,P39679,HTH_APSES,MBP1_KLULA,6,112,20.833,IYSAKYSGVDVYEFIHPTGSIMKRKADNWVNATHILKAAKFPKAKR...,IYSAKYSGVDVYEFIHP--..-TGSIMKRKADNWVNATHILKAAKF...
1,PS51299,P33520,HTH_APSES,RES1_SCHPO,6,112,18.223,IHKITYSGVEVFEYTINGFPLMKRCHDNWLNATQILKIAELDKPRR...,IHKITYSGVEVFEYTI---..NGFPLMKRCHDNWLNATQILKIAEL...
2,PS51299,P36011,HTH_APSES,STUA_EMENI,129,235,18.080,RVTATLWEDEGSLCYQVEAKGVCVARREDNGMINGTKLLNVAGMTR...,RVTATLWEDEGSLCYQVEA..KGVCVARREDNGMINGTKLLNVAGM...
3,PS51299,P36093,HTH_APSES,PHD1_YEAST,186,292,16.870,RVITTMWEDENTICYQVEANGISVVRRADNNMINGTKLLNVTKMTR...,RVITTMWEDENTICYQVEA..NGISVVRRADNNMINGTKLLNVTKM...
4,PS51299,Q4X228,HTH_APSES,STUA_ASPFU,129,235,17.962,RVTATLWEDEGSLCYQVEAKGVCVARREDNHMINGTKLLNVAGMTR...,RVTATLWEDEGSLCYQVEA..KGVCVARREDNHMINGTKLLNVAGM...
...,...,...,...,...,...,...,...,...,...
620070,PS51486,Q8IVW6,REKLES,ARI3B_HUMAN,419,517,32.801,AALEQLRERLESGEPAEKKASRLSEEEQRLVQQAFQRNFFSMARQL...,AALEQLRERLESGEPAEKKASR........LSEEEQRLVQQAFQRN...
620071,PS51486,Q24573,REKLES,DRI_DROME,731,825,26.294,TTGGSVGHRHSSPVSTKKKGGAKPQSGGKDVPTEDKDASSSGKLNP...,TTGGSVGHRHSSPVSTKKKGGA........KPQSGGKDVPTEDK-D...
620072,PS51486,Q6GQD7,REKLES,ARI3A_XENLA,404,499,33.184,AALEQLREKLESGEPPEKKMALGSEEQQRIIQRTIQHNLLAMTAQL...,AALEQLREKLESGEPPEKKMAL........GSEEQQRIIQRTIQHN...
620073,PS51486,A6PWV5,REKLES,ARI3C_MOUSE,301,386,21.411,LASEATREKLAPEEPPEKRAVLMGPVDSPRLGAPPSFLPRGKAPLR...,LASEATREKLAPEEPPEKRAVL........MGPVDSPRLGAPPS--...


In [ ]:
pd.read_sql(f"SELECT * FROM prosite_go_mapping", conn)

,prosite_id,prosite_description,go_id,go_name
0,PS00011,GLA_1,GO:0005509,calcium ion binding
1,PS00011,GLA_1,GO:0005576,extracellular region
2,PS00026,CHIT_BIND_I_1,GO:0008061,chitin binding
3,PS00027,HOMEOBOX_1,GO:0000981,"DNA-binding transcription factor activity, RNA..."
4,PS00027,HOMEOBOX_1,GO:0006355,regulation of DNA-templated transcription
...,...,...,...,...
2808,PS60025,CONANTOKIN,GO:0035792,host cell postsynaptic membrane
2809,PS60026,ERGTX,GO:0019870,potassium channel inhibitor activity
2810,PS60026,ERGTX,GO:0005576,extracellular region
2811,PS60028,SCORPION_CALCINE,GO:0019855,calcium channel inhibitor activity


In [13]:
conn.close()